# オプティマイズ
この版はパラメトリック最適化にするつもり。
- 中心周波数
- バンド幅
- 分散補償
計 3パラメーター

必要なものは
- スクイージングレベルの測定
    - ポンプ光のON/OFF arduinoモジュール OK
    - スペアナのSCPIコントロール
    - もしかしたらロック角度の変調
- waveshaperコントロール
- パラメーター生成

In [1]:
import numpy as np
from convinience import Spectrum, Band, synthesisSpectrum
import copy

In [ ]:
# 各種設定周り

class Setting:
    def __init__(self):
        self.ws_ip                  = '169.254.6.8'
        self.ws_Freq_resolution     = 0.001 # 0.001THz = 1GHz
        self.ws_Freqrange           = None
        self.optSpectrum : Spectrum = None

    def condition_approval(self):
        assert self.ws_Freqrange             is not None
        assert self.optSpectrum              is not None
        assert self.optSpectrum.powerdensity is not None
        assert self.optSpectrum.frequency    is not None
        assert self.optSpectrum.wavelength   is not None
        assert self.optSpectrum.phase        is not None

setting = Setting()

from WSMethods import getWSrange
startFreq, endFreq    = getWSrange(setting.ws_ip)
setting.ws_Freqrange  = np.arange(startFreq, endFreq, setting.ws_Freq_resolution)

setting.optSpectrum.band.frequency = np.load('optFreq.npy')
setting.optSpectrum.powerdensity   = np.load('optSpectrum.npy')
setting.optSpectrum.phase          = np.zeros_like(setting.optSpectrum.band.frequency)

setting.condition_approval()

In [ ]:
def gaussian_profile(centerWavelength = 1545.2, bandWidth = 0.3, band : Band= None, dispersion = 0):
    c = 299792458
    spectrum = Spectrum()
    spectrum.band = copy.deepcopy(band)
    spectrum.powerdensity = -10*((band.wavelength*1E9 - centerWavelength)/(bandWidth/2.0))**2 * 0.3 # dB
    centerFreq = c/(centerWavelength*1E-9)/1E12
    spectrum.phase = (centerWavelength*1E-9)**2/(1*np.pi*c)*(dispersion*1E-2)*((band.frequency-centerFreq)*1E12)**2
    return spectrum

def compensation(optSpectrum: Spectrum, targetSpectrum: Spectrum, band: Band):
    tempSpectrum = copy.deepcopy(optSpectrum)
    tempSpectrum.powerdensity = -tempSpectrum.powerdensity
    new_spectrum = synthesisSpectrum(tempSpectrum, targetSpectrum, band)
    # たとえば1540~1550nm帯域で増幅(new_spectrum.powerdensity)しないように、引いとくとか。
    range_max = np.max(new_spectrum.powerdensity[(1540 < new_spectrum.wavelength) & (new_spectrum.wavelength < 1550)])
    new_spectrum.powerdensity -= range_max
    return new_spectrum